In [1]:
import numpy as np
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
import IPython
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Reshape,MaxPooling2D, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, models, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tqdm import tqdm

In [2]:
base_path = '/kaggle/input/asvpoof-2019-dataset/LA/LA'
protocol_dir = os.path.join(base_path, 'ASVspoof2019_LA_cm_protocols')
train_dir = os.path.join(base_path, 'ASVspoof2019_LA_train', 'flac')
dev_dir = os.path.join(base_path, 'ASVspoof2019_LA_dev', 'flac')
eval_dir = os.path.join(base_path, 'ASVspoof2019_LA_eval', 'flac')

In [3]:
def get_file_path(directory, filename):
    return os.path.join(directory, f'{filename}.flac')
def read_dataset(protocol_path, directory):
    """Reads the dataset from a protocol file and returns a DataFrame."""
    df = pd.read_csv(protocol_path, sep=' ', header=None, names=['speaker_id', 'filename', 'system_id', 'null', 'class_name'])
    df['filepath'] = df['filename'].apply(lambda x: get_file_path(directory, x))
    df.drop('null', axis=1, inplace=True)
    df.dropna(inplace=True)
    return df

def label_to_int(class_name):
    if class_name == 'bonafide':
        return 0
    else:
        return 1

def add_columns(df, subset):
    df['target'] = df['class_name'].apply(label_to_int)
    df['subset'] = subset
    return df

def sample_data(df, n_bonafide, n_spoof):
    bonafide = df[df['class_name'] == 'bonafide'].head(n_bonafide)
    spoof = df[df['class_name'] != 'bonafide'].head(n_spoof)
    return pd.concat([bonafide, spoof])

In [4]:
train_df = read_dataset(os.path.join(protocol_dir, 'ASVspoof2019.LA.cm.train.trn.txt'), train_dir)
dev_df = read_dataset(os.path.join(protocol_dir, 'ASVspoof2019.LA.cm.dev.trl.txt'), dev_dir)
eval_df = read_dataset(os.path.join(protocol_dir, 'ASVspoof2019.LA.cm.eval.trl.txt'), eval_dir)

train_df = add_columns(train_df, 'train')
dev_df = add_columns(dev_df, 'dev')
eval_df = add_columns(eval_df, 'eval')

In [5]:
data_df = pd.concat([train_df, dev_df, eval_df], ignore_index=True)
data_df

,speaker_id,filename,system_id,class_name,filepath,target,subset
0,LA_0079,LA_T_1138215,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0,train
1,LA_0079,LA_T_1271820,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0,train
2,LA_0079,LA_T_1272637,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0,train
3,LA_0079,LA_T_1276960,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0,train
4,LA_0079,LA_T_1341447,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0,train
...,...,...,...,...,...,...,...
121456,LA_0004,LA_E_1665632,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0,eval
121457,LA_0038,LA_E_5085671,-,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1,eval
121458,LA_0012,LA_E_4926022,-,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1,eval
121459,LA_0052,LA_E_2894498,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0,eval


In [6]:
!pip install spafe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 1.2 MB/s eta 0:00:00


In [7]:
from spafe.features.cqcc import cqcc

def extract_cqcc(audio, sr, n_cqcc=200):
    # Gọi hàm cqcc từ thư viện spafe
    cqcc_features = cqcc(audio, fs=sr, num_ceps=n_cqcc)
    return cqcc_features

# Load file âm thanh
audio_path = data_df['filepath'][1]
audio, sr = librosa.load(audio_path, sr=None)

# Trích xuất CQCC
cqcc_features = extract_cqcc(audio, sr)
print("CQCC Features Shape:", cqcc_features.shape)

CQCC Features Shape: (438, 70)


In [8]:
tqdm.pandas()

def extract_features(file_path, max_length=100, n_mfcc=100):
    audio, sr = librosa.load(file_path, sr=16000)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)  # Reduced number for efficiency

    # Extract Mel_spectrogram
    cqcc_ = cqcc(audio, fs=sr, num_ceps=n_mfcc)

    # Load the audio file (replace 'your_audio_file.wav' with your file)
    audio_path = data_df['filepath'][0]
    y, sr = librosa.load(audio_path, sr=None)  # y is the audio time series, sr is the sampling rate

    # Calculate the short-time Fourier transform (STFT) to get the spectrogram
    D = np.abs(librosa.stft(y))

    # Convert amplitude spectrogram to decibel units
    DB = librosa.amplitude_to_db(D, ref=np.max)
    

    # Ensure all features have the same time dimension
    min_length = min(mfccs.shape[1], DB.shape[1], cqcc_.shape[1])
    if min_length < max_length:
        mfccs = np.pad(mfccs, ((0,max(0,max_length - mfccs.shape[0])), (0, max(0,max_length - mfccs.shape[1]))), mode='constant')
        cqcc_ = np.pad(cqcc_, ((0, max(0,max_length - cqcc_.shape[0])), (0, max(0,max_length - cqcc_.shape[1]))), mode='constant')
        DB = np.pad(DB, ((0,max(0,max_length - DB.shape[0])), (0, max(0,max_length - DB.shape[1]))), mode='constant')
   
    mfccs = mfccs[:n_mfcc, :max_length]
    cqcc_ = cqcc_[:n_mfcc, :max_length]
    DB = DB[:n_mfcc, :max_length]
        
    combined_features = np.stack([mfccs, cqcc_, DB], axis=-1)

    return combined_features

# Assuming data_df is a DataFrame and 'filepath' is a column with audio file paths
features = data_df['filepath'].progress_apply(extract_features)

 97%|█████████▋| 117466/121461 [5:35:56<12:28,  5.34it/s]

In [ ]:
X = np.array(data_df['feature'].tolist())
np.save('/kaggle/working/feature_LA_MFCC_CQCC_Spectrogram.npy', X)